In [1]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [2]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [3]:
sheet = sa.open('Wani Spreadsheet')

In [4]:
database = sheet.worksheet('Database Harga')

In [13]:
df_harga = pd.DataFrame(database.get_all_records())

In [56]:
worksheet = sheet.worksheet('Pesanan')

In [57]:
df = pd.DataFrame(worksheet.get_all_records())

In [58]:
df.rename(columns={'Request cabe GEPREK': 'Cabe'}, inplace=True)

In [59]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,13/04/2023 08:34:24,Rey,Indomie Telur Geprek,0,,Es teh,
1,13/04/2023 08:34:54,Rey,Indomie Ayam Geprek,1,,,
2,13/04/2023 08:37:41,Tot,Nasi Ayam Geprek,2,,Es teh,
3,13/04/2023 08:43:12,JoWi,Nasi Ayam Geprek,,Tahu Crispy tanpa bumbu,,
4,13/04/2023 09:07:59,Max,Nasi Ayam Geprek,,,,


In [60]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1
df[list] = df[list].replace('', 1)

df[list]

,Makanan Utama,Cabe,Nama
0,Indomie Telur Geprek,0,Rey
1,Indomie Ayam Geprek,1,Rey
2,Nasi Ayam Geprek,2,Tot
3,Nasi Ayam Geprek,1,JoWi
4,Nasi Ayam Geprek,1,Max
5,Nasi Goreng Spesial,1,Pacar e Pak Evan
6,Indomie Ayam Geprek,1,Justin
7,Nasi Goreng Kulit,1,Vania apple
8,Nasi Ayam Mentega,1,Andreas
9,Indomie Telur Geprek,2,Bat


In [61]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek') & ~df['Makanan Utama'].str.contains('Indomie')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
2,13/04/2023 08:37:41,Tot,Nasi Ayam Geprek,2,,Es teh,
3,13/04/2023 08:43:12,JoWi,Nasi Ayam Geprek,1,Tahu Crispy tanpa bumbu,,
4,13/04/2023 09:07:59,Max,Nasi Ayam Geprek,1,,,


In [62]:
# Dataframe non-geprek
df_not_geprek = df[~df['Makanan Utama'].str.contains('Geprek')]

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [63]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-04-13 09:43:11.766425
1 Nasi Ayam Geprek Cabe 2 (Tot)
2 Nasi Ayam Geprek Cabe 1 (JoWi, Max)
1 Nasi Goreng Spesial (Pacar e Pak Evan)
1 Nasi Goreng Kulit (Vania apple)
1 Nasi Ayam Mentega (Andreas)
1 Indomie Telur Geprek Cabe 0 (Rey)
2 Indomie Ayam Geprek Cabe 1 (Rey, Justin)
1 Indomie Telur Geprek Cabe 2 (Bat)
1 Tahu Crispy tanpa bumbu (JoWi)
1 Kulit Ayam Crispy (Bat)
2 Es teh (Rey, Tot)


In [64]:
# remove all text after '(' in combined_prints
# combined_prints = [i.split('(')[0] for i in combined_prints]

# print('\n'.join(combined_prints))

In [65]:
payload = {'message' : '\n'.join(combined_prints)}
r = requests.post('https://notify-api.line.me/api/notify'
                , headers={'Authorization' : 'Bearer {}'.format(token)}
                , params = payload)